# Document AI Synchronous API
This notebook shows you how use Python to make synchronous calls to the Document AI API

You must replace the `processor_id` variable value in the the second cell with the appropriate value for the Processor ID of the Document AI processor that you want to use. The processor may not support all of the Document AI output properties. Entity data is only returned by processors that use specialized parsers for example. 

In [1]:
from google.cloud import documentai_v1beta3 as documentai
from google.cloud import storage
from prettytable import PrettyTable

project_id=%system gcloud config get-value core/project
project_id=project_id[0]
location = 'us'           # Replace with 'eu' if processor does not use 'us' location
file_path = 'form.pdf'    # Replace this with the name of the pdf file to process if necessary


In [3]:
# Set your Processor ID
processor_id = 'e4d43d681050128a' # TODO: Replace with a valid Processor ID 

In [4]:
# Process Document Function
def process_document(
        project_id=project_id, location=location, 
        processor_id=processor_id,  file_path=file_path
):
    # Instantiates a client
    client = documentai.DocumentProcessorServiceClient()
    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"
    with open(file_path, "rb") as image:
        image_content = image.read()
    # Read the file into memory
    document = {"content": image_content, "mime_type": "application/pdf"}
    # Configure the process request
    request = {"name": name, "document": document}
    # Use the Document AI client to process the sample form
    result = client.process_document(request=request)
    return result.document

In [ ]:
# Please like share and subscribe to Techcps
# https://www.youtube.com/@techcps

print("please like share and subscribe to techcps https://www.youtube.com/@techcps")

In [5]:
# Process Document 
document=process_document()
# print all detected text. 
# All document processors will display the text content
print("Document processing complete.")
print("Text: {}".format(document.text))

Document processing complete.
Text: FakeDoc M.D.
HEALTH INTAKE FORM
Please fill out the questionnaire carefully. The information you provide will be used to complete
your health profile and will be kept confidential.
Date:
9/14/19
Name:
Sally Walker
DOB: 09/04/1986
Address: 24 Barney Lane
City: Towaco
State: NJ Zip: 07082
Email: Sally, walker@cmail.com
_Phone #: (906) 917-3486
Gender: F
Marital Status:
Single Occupation: Software Engineer
Referred By: None
Emergency Contact: Eva Walker Emergency Contact Phone: (906)334-8926
Describe your medical concerns (symptoms, diagnoses, etc):
Ranny nose, mucas in thwat, weakness,
aches, chills, tired
Are you currently taking any medication? (If yes, please describe):
Vyvanse (25mg) daily for attention.



In [6]:
# Get Text Function
# Define a function to retrieve an object dictionary for a named element
def get_text(doc_element: dict, document: dict):
    """
    Document AI identifies form fields by their offsets
    in document text. This function converts offsets
    to text snippets.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in doc_element.text_anchor.text_segments:
        start_index = (
            int(segment.start_index)
            if segment in doc_element.text_anchor.text_segments
            else 0
        )
        end_index = int(segment.end_index)
        response += document.text[start_index:end_index]
    return response

In [7]:
# Display Form Data 
# Form data is only detected by some processors such as those that use the General Form parser.
document_pages = document.pages
print("Form data detected:\n")
# For each page fetch each form field and display fieldname, value and confidence scores
for page in document_pages:
    print("Page Number:{}".format(page.page_number))
    for form_field in page.form_fields:
        fieldName=get_text(form_field.field_name,document)
        nameConfidence = round(form_field.field_name.confidence,4)
        fieldValue = get_text(form_field.field_value,document)
        valueConfidence = round(form_field.field_value.confidence,4)
        print(fieldName+fieldValue +"  (Confidence Scores: (Name) "+str(nameConfidence)+", (Value) "+str(valueConfidence)+")\n")


Form data detected:

Page Number:1
Are you currently taking any medication? (If yes, please describe):
Vyvanse (25mg) daily for attention.
  (Confidence Scores: (Name) 0.9433, (Value) 0.9433)

_Phone #: walker@cmail.com
(906  (Confidence Scores: (Name) 0.9323, (Value) 0.9323)

Zip: 07082
  (Confidence Scores: (Name) 0.9142, (Value) 0.9142)

City: Towaco
  (Confidence Scores: (Name) 0.9005, (Value) 0.9005)

State: NJ   (Confidence Scores: (Name) 0.8939, (Value) 0.8939)

DOB: 09/04/1986
  (Confidence Scores: (Name) 0.8852, (Value) 0.8852)

Gender: F
  (Confidence Scores: (Name) 0.8824, (Value) 0.8824)

Name:
Sally Walker
  (Confidence Scores: (Name) 0.8728, (Value) 0.8728)

Marital Status:
Single   (Confidence Scores: (Name) 0.8524, (Value) 0.8524)

Describe your medical concerns (symptoms, diagnoses, etc):
Ranny nose, mucas in thwat, weakness,
aches, chills, tired
  (Confidence Scores: (Name) 0.8439, (Value) 0.8439)

Date:
9/14/19
  (Confidence Scores: (Name) 0.83, (Value) 0.83)

Addres

In [8]:
# Display Entity Data
# Entity data is only detected by specialized parsers, such as the Procurement Expense parser.
# For each entity print the key/value pair and their corresponding confidence scores.
if 'entities' in dir(document):
    entities=document.entities
    table = PrettyTable(['Type', 'Value', 'Confidence'])
    entities_found = 0
    for entity in entities:
       entity_type = entity.type_
       value = entity.mention_text
       confidence = round(entity.confidence,4)
       table.add_row([entity_type, value, confidence])
    print(table)       
else:
    print("Document does not contain entity data.")

+------------------+-------+------------+
|       Type       | Value | Confidence |
+------------------+-------+------------+
| generic_entities |       |    0.0     |
+------------------+-------+------------+
